In [1]:
from ctypes.wintypes import RGB
import cv2
import mediapipe as mp

2023-08-13 11:22:51.321293: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-13 11:22:51.322570: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-13 11:22:51.352718: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-13 11:22:51.353300: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-13 11:22:52.061968: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
mp_drawing = mp.solutions.drawing_utils 
mp_drawing_styles = mp.solutions.drawing_styles 
mp_pose = mp.solutions.pose 

In [3]:
cap = cv2.VideoCapture(-1)

with mp_pose.Pose (min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as pose:
    while cap.isOpened():
        success,image = cap.read()
        if not success:
            print("ignoring No video in camera frame")
            continue
        
        
        image.flags.writeable = False
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        
        results = pose.process(image)
        
        image.flags.writeable = True
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
        
        mp_drawing.draw_landmarks(
            image,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec = mp_drawing_styles.get_default_pose_landmarks_style()
        )
        
        cv2.imshow('MediaPipe Pose Estimation ',cv2.flip(image,1))
        
        if cv2.waitKey(5) & 0xFF == 27:
            break
        

cv2.destroyAllWindows()

cap.release()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


### Storing the pose Landmarks

In [2]:
import cv2
import mediapipe as mp
import pickle

mp_pose = mp.solutions.pose 

cap = cv2.VideoCapture(-1)

pose_landmarks_history = []

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring: No video in camera frame")
            continue
        
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        results = pose.process(image)
        
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.pose_landmarks is not None:
            pose_landmarks_history.append(results.pose_landmarks)
        
        cv2.imshow('MediaPipe Pose Estimation', cv2.flip(image, 1))
        
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

cv2.destroyAllWindows()
cap.release()

# Save pose_landmarks_history using pickle
with open('pose_landmarks_history.pkl', 'wb') as f:
    pickle.dump(pose_landmarks_history, f)


2023-08-13 11:30:08.415774: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-13 11:30:08.417210: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-13 11:30:08.449869: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-13 11:30:08.450375: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-13 11:30:09.178164: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

### To replay the landmarks 

In [3]:
import cv2
import mediapipe as mp
import pickle
import numpy as np

mp_drawing = mp.solutions.drawing_utils 
mp_drawing_styles = mp.solutions.drawing_styles 

# Load the stored pose landmarks
with open('pose_landmarks_history.pkl', 'rb') as f:
    pose_landmarks_history = pickle.load(f)

for landmarks in pose_landmarks_history:
    image = 255 * np.ones((480, 640, 3), dtype=np.uint8)  # Blank white image

    mp_drawing.draw_landmarks(
        image,
        landmarks,
        mp.solutions.pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style()
    )

    cv2.imshow('Motion Visualization', cv2.flip(image, 1))

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()


NameError: name 'np' is not defined